In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine



SyntaxError: invalid syntax (1048225456.py, line 1)

In [ ]:
#Connection with MySQL Database
user = 'leonardo'
passw = 'senha'
host =  '127.0.0.1'  # either localhost or ip e.g. '172.17.0.2' or hostname address 
port = 3306 
database = 'nubank_test'

mydb = create_engine('mysql+pymysql://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)

In [2]:
#Read csv files for each table
df_accounts = pd.read_csv("part-00000-tid-2834924781296170616-a9b7a53c-b8f1-417c-876b-22ce8ab4c825-11024507-1-c000.csv")
df_city = pd.read_csv("part-00000-tid-7257851286237664629-906ff0c6-51c7-4cdd-97cd-dc0aa92a92f1-11185485-1-c000.csv")
df_country = pd.read_csv("part-00000-tid-5054581782199228501-41100236-b09c-46a7-8442-8d968c697e9a-11286346-1-c000.csv")
df_customers = pd.read_csv("part-00000-tid-2581180368179082894-038c4dab-1615-444a-994f-5a2107c4d9a8-11135734-1-c000.csv")
df_d_month = pd.read_csv("part-00000-tid-1411108534556725179-0fbf435f-dd8b-411e-ae97-11e74b345b4f-10859988-1-c000.csv")
df_d_time = pd.read_csv("part-00000-tid-2902183611695287462-3111e3e9-023d-4533-a7e5-d0b87fb1a808-10662337-1-c000.csv")
df_d_week = pd.read_csv("part-00000-tid-8944900144231406848-ea4ff673-508c-46e0-ac10-e488037f37c5-10733320-1-c000.csv")
df_d_weekday = pd.read_csv("part-00000-tid-1198883628307677355-d7f0c6f5-b5f5-40c6-9537-2951b88849eb-10926413-1-c000.csv")
df_d_year = pd.read_csv("part-00000-tid-5440254676189819830-69d9a28f-dbd2-48ca-9eca-66f1ddb276df-10961064-1-c000.csv")
df_pix_moviments = pd.read_csv("part-00000-tid-8322739320471544484-12382b61-f87b-4388-931d-ec1681d2aad1-10545794-1-c000.csv")
df_state = pd.read_csv("part-00000-tid-2467726635302089596-e4fdd6ee-8624-4658-af6a-6098bc1c0825-11243350-1-c000.csv")
df_transfer_ins = pd.read_csv("part-00000-tid-3939740088886661710-b68a1bdc-ca76-4934-b2fd-756b973d041f-10414417-1-c000.csv")
df_transfer_outs = pd.read_csv("part-00000-tid-3880462020784336524-8a5c83e1-e1e4-471e-9dbf-1b37babaff47-10468383-1-c000.csv")

In [3]:
df_d_time.head()

,time_id,action_timestamp,week_id,month_id,year_id,weekday_id
0,1593136311,2020-06-26T01:51:51.000Z,102426675,3042,1024140,2028
1,1596889526,2020-08-08T12:25:26.000Z,102429717,4056,1024140,2535
2,1601172264,2020-09-27T02:04:24.000Z,102433266,4563,1024140,3042
3,1596544748,2020-08-04T12:39:08.000Z,102429717,4056,1024140,507
4,1603372558,2020-10-22T13:15:58.000Z,102435294,5070,1024140,1521


In [4]:
#Adjusting investments file in different format
df_investments_json = pd.read_json("investments_json.txt")
df_investments_json.head()

,account_id,transactions
0,623509224135263,"[{'transaction_id': '832358148869144320', 'sta..."
1,1910868644230470,"[{'transaction_id': '555441156782091328', 'sta..."
2,1963067196365885,"[{'transaction_id': '2986130373467638784', 'st..."
3,1977223077647860,"[{'transaction_id': '3100752537337675264', 'st..."
4,3417018705901279,"[{'transaction_id': '427850048417224768', 'sta..."


In [5]:
account_id = [2980018541051129344, 2014046747593812480, 2803803553757944832, 1339139825131710464, 871859688032364672, 
              1871764561957751808, 930886615797514368, 905754416586163200, 752997682765132288, 2937213920413407744]
df_investments_json = df_investments_json[df_investments_json["account_id"].isin(account_id)]
#df_investments_json[df_investments_json["account_id"]==2014046747593812480]

In [6]:
df_investments_json = df_investments_json.reset_index(drop=True)

In [7]:
df = pd.json_normalize(df_investments_json['transactions'])

In [ ]:
for column in df:
    print(column)

In [8]:
#Columns trasactions for investments table in json format, normalize and concatenation with account_id
df_inv0 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[0])], axis=1)
df_inv1 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[1])], axis=1)
df_inv2 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[2])], axis=1)
df_inv3 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[3])], axis=1)
df_inv4 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[4])], axis=1)
df_inv5 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[5])], axis=1)
df_inv6 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[6])], axis=1)
df_inv7 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[7])], axis=1)
df_inv8 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[8])], axis=1)
df_inv9 = pd.concat([df_investments_json["account_id"],pd.json_normalize(pd.json_normalize(df_investments_json['transactions'])[9])], axis=1)

In [9]:
#Concatenation all transaction in list for each account_id in investment table
df_inv = pd.concat([df_inv0, df_inv1, df_inv2, df_inv3, df_inv4, df_inv5, df_inv6, df_inv7, df_inv8, df_inv9], ignore_index=True)

In [10]:
df_inv

,account_id,transaction_id,status,amount,investment_requested_at,investment_completed_at,investment_completed_at_timestamp,type
0,752997682765132288,2633932355582452736,completed,992.51,1600896022,1600896031,2020-09-23T21:20:31.000000000,investment_transfer_in
1,871859688032364672,1906974999589947904,completed,824.07,1606434885,1606434887,2020-11-26T23:54:47.000000000,investment_transfer_in
2,905754416586163200,2857739979614177792,completed,462.56,1601117107,1601117112,2020-09-26T10:45:12.000000000,investment_transfer_in
3,930886615797514368,811572868142792960,completed,1455.69,1603886241,1603886254,2020-10-28T11:57:34.000000000,investment_transfer_in
4,1339139825131710464,2106144186981122304,completed,291.15,1596545138,1596545146,2020-08-04T12:45:46.000000000,investment_transfer_out
...,...,...,...,...,...,...,...,...
95,1871764561957751808,936831370616722048,completed,707.31,1580082714,1580082716,2020-01-26T23:51:56.000000000,investment_transfer_out
96,2014046747593812480,260250899601279712,completed,717.42,1589894382,1589894399,2020-05-19T13:19:59.000000000,investment_transfer_in
97,2803803553757944832,641723321248566784,completed,1160.47,1603242915,1603242927,2020-10-21T01:15:27.000000000,investment_transfer_in
98,2937213920413407744,2231896346368720384,completed,730.16,1584815548,1584815551,2020-03-21T18:32:31.000000000,investment_transfer_out


In [11]:
#Removing nullable transaction_id in investiments table
df_inv = df_inv[df_inv['transaction_id'].notna()]
df_inv = df_inv.reset_index(drop=True)

In [ ]:
df_investments = df_inv[['transaction_id', 'account_id', 'type', 'amount', 'investment_requested_at', 'investment_completed_at', 'status']]

In [ ]:
mydb.execute('drop table if exists investments')

In [ ]:
df_investments.to_sql('investments', mydb, index=False)

In [ ]:
#Adjusting column sorting for each table that were different according to model
df_country = df_country[['country_id', 'country']]
df_city = df_city[['city_id', 'city', 'state_id']]
df_customers = df_customers[['customer_id','first_name','last_name', 'customer_city', 'country_name', 'cpf']]
df_pix_moviments = df_pix_moviments[['id', 'account_id', 'in_or_out', 'pix_amount', 'pix_requested_at', 'pix_completed_at', 'status']]
df_state = df_state[['state_id', 'state', 'country_id']]
df_investments = df_inv[['transaction_id', 'account_id', 'type', 'amount', 'investment_requested_at', 'investment_completed_at', 'status']]

In [ ]:
#Drop table if exists to recreate
mydb.execute('drop table if exists accounts')
mydb.execute('drop table if exists city')
mydb.execute('drop table if exists country')
mydb.execute('drop table if exists customers')
mydb.execute('drop table if exists d_month')
mydb.execute('drop table if exists d_time')
mydb.execute('drop table if exists d_week')
mydb.execute('drop table if exists d_weekday')
mydb.execute('drop table if exists d_year')
mydb.execute('drop table if exists investments')
mydb.execute('drop table if exists pix_moviments')
mydb.execute('drop table if exists state')
mydb.execute('drop table if exists transfer_ins')
mydb.execute('drop table if exists transfer_outs')

In [ ]:
#Writing tables with dataframes
df_accounts.to_sql('accounts', mydb, index=False)
df_city.to_sql('city', mydb, index=False)
df_country.to_sql('country', mydb, index=False)
df_customers.to_sql('customers', mydb, index=False)
df_d_month.to_sql('d_month', mydb, index=False)
df_d_time.to_sql('d_time', mydb, index=False)
df_d_week.to_sql('d_week', mydb, index=False)
df_d_weekday.to_sql('d_weekday', mydb, index=False)
df_d_year.to_sql('d_year', mydb, index=False)
df_investments.to_sql('investments', mydb, index=False)
df_pix_moviments.to_sql('pix_moviments', mydb, index=False)
df_state.to_sql('state', mydb, index=False)
df_transfer_ins.to_sql('transfer_ins', mydb, index=False)
df_transfer_outs.to_sql('transfer_outs', mydb, index=False)

In [ ]:
qry = '''
        WITH pix AS ( 
            select
                 y.action_year
                ,m.action_month
                ,c.customer_id
                ,case when p.in_or_out = 'pix_in' then p.pix_amount else 0 end as transfer_in 
                ,case when p.in_or_out = 'pix_out' then p.pix_amount else 0 end as transfer_out  
            from
                nubank_test.pix_moviments p
                inner join nubank_test.accounts a on p.account_id = a.account_id
                inner join nubank_test.customers c on a.customer_id = c.customer_id
                inner join nubank_test.d_time t on p.pix_completed_at = t.time_id
                inner join nubank_test.d_month m on t.month_id = m.month_id
                inner join nubank_test.d_year y on t.year_id = y.year_id
            where
                p.status = 'completed'
                and y.action_year = 2020
            ),
        ins AS (
            select
                 y.action_year
                ,m.action_month
                ,c.customer_id
                ,p.amount as transfer_in
                ,0 transfer_out
            from
                nubank_test.transfer_ins p
                inner join nubank_test.accounts a on p.account_id = a.account_id
                inner join nubank_test.customers c on a.customer_id = c.customer_id
                inner join nubank_test.d_time t on p.transaction_completed_at = t.time_id
                inner join nubank_test.d_month m on t.month_id = m.month_id
                inner join nubank_test.d_year y on t.year_id = y.year_id
            where
                p.status = 'completed'
                and y.action_year = 2020
        ),
        outs AS (
            select
                 y.action_year
                ,m.action_month
                ,c.customer_id
                ,0 as transfer_in
                ,p.amount as transfer_out
            from
                nubank_test.transfer_outs p
                inner join nubank_test.accounts a on p.account_id = a.account_id
                inner join nubank_test.customers c on a.customer_id = c.customer_id
                inner join nubank_test.d_time t on p.transaction_completed_at = t.time_id
                inner join nubank_test.d_month m on t.month_id = m.month_id
                inner join nubank_test.d_year y on t.year_id = y.year_id
            where
                p.status = 'completed'
                and y.action_year = 2020
        ),
        union_qry AS
        (
            select * from pix
            union all
            select * from ins
            union all
            select * from outs
        )
        select action_month as "Month"
             , customer_id as "Customer"
             , ROUND(SUM(transfer_in), 2) as "Total Transfer In"
             , ROUND(SUM(transfer_out), 2) as "Total Transfer Out"
             , ROUND(SUM(COALESCE(sum(transfer_in)-sum(transfer_out), 0)) OVER(PARTITION BY customer_id ORDER BY action_month), 2) as "Account Monthly Balance"
        from union_qry
        group by action_month
             , customer_id
        order by 1, 2
    '''
df_statement_1 = pd.read_sql_query(qry, mydb)

In [ ]:
df_statement_1.head()

In [ ]:
df_statement_1.to_csv(r"df_statement_1.csv", index=False, sep=";")

In [ ]:
df_accounts.to_csv(r"account.csv", index=False, sep=";")
df_city.to_csv(r"city.csv", index=False, sep=";")
df_country.to_csv(r"country.csv", index=False, sep=";")
df_customers.to_csv(r"customers.csv", index=False, sep=";")
df_d_month.to_csv(r"d_month.csv", index=False, sep=";")
df_d_time.to_csv(r"d_time.csv", index=False, sep=";")
df_d_week.to_csv(r"d_week.csv", index=False, sep=";")
df_d_weekday.to_csv(r"d_weekday.csv", index=False, sep=";")
df_d_year.to_csv(r"d_year.csv", index=False, sep=";")
df_investments.to_csv(r"investments.csv", index=False, sep=";")
df_pix_moviments.to_csv(r"pix_moviments.csv", index=False, sep=";")
df_state.to_csv(r"state.csv", index=False, sep=";")
df_transfer_ins.to_csv(r"transfer_ins.csv", index=False, sep=";")
df_transfer_outs.to_csv(r"transfer_outs.csv", index=False, sep=";")

In [ ]:
qry = '''
    select
         y.action_year
        ,m.action_month
        ,c.customer_id
        ,sum(p.amount) as transfer_in
        ,0 transfer_out
    from
        transfer_ins p
        inner join accounts a on p.account_id = a.account_id
        inner join customers c on a.customer_id = c.customer_id
        inner join d_time t on p.transaction_completed_at = t.time_id
        inner join d_month m on t.month_id = m.month_id
        inner join d_year y on t.year_id = y.year_id
    where
        p.status = 'completed'
        and y.action_year = 2020
    group by c.customer_id
            ,m.action_month
            ,y.action_year
    '''
df_teste = pd.read_sql_query(qry, conn)
df_teste

In [ ]:
qry = '''
        WITH one AS ( SELECT 1 )
        SELECT * FROM one;
    '''
df_teste = pd.read_sql_query(qry, conn)
df_teste

In [ ]:
qry = '''
    select  
        *
    from
        transfer_ins
    limit 10
    '''
df_teste = pd.read_sql_query(qry, conn)
df_teste

In [ ]:
qry = '''
    select  
        *
    from
        accounts 
    '''
df_teste = pd.read_sql_query(qry, conn)
df_teste

In [ ]:
mydb

In [ ]:
df_transfer_ins.to_sql('transfer_ins1', mydb, index=False)

In [ ]:
df_d_time.to_sql('d_time', connection, index=False)